In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env.notebook
# This allows you to keep your notebook-specific secrets and configurations separate
# from your main application's .env file.
if os.path.exists("../.env.notebook"):
    load_dotenv(dotenv_path="../.env.notebook")


### Overview
This notebook orchestrates a two-stage AI workflow:
1. Review each patient's clinical JSON to decide if preventive outreach is needed and (if so) generate a structured `CALL_BRIEF`.
2. Generate a concise, natural phone message from a selected `CALL_BRIEF`, then optionally place an outbound call via Azure Communication Services.

The next cell only contains (commented) optional install commands for dependencies; it's intentionally lightweight so environments that already have packages aren't slowed down.

In [ ]:
# If your environment doesn't have these already, uncomment:
#!pip install --quiet --upgrade openai pandas
#!pip install --upgrade "azure-communication-callautomation>=1.2.0"
#!pip install --quiet azure-communication-callautomation fastapi uvicorn nest_asyncio pyngrok

#import pkg_resources
#print("callautomation version:", pkg_resources.get_distribution("azure-communication-callautomation").version)

### Imports, Environment, and Configuration
The next cell:
- Imports core libraries (`os`, `json`, `re`, `pandas`, path utilities, typing helpers).
- Loads required environment variables for Azure OpenAI + Azure Communication Services + TTS.
- Prints a quick readiness report (which secrets are set) to help validate runtime config before any API calls.
No external network calls happen yet—it's pure setup / validation.

In [ ]:
import os, json, re
from pathlib import Path
from typing import Any, Dict, List, Optional

import pandas as pd

# Env vars (set these in your environment or just override here)

NOTES_PATH = os.getenv("NOTES_PATH", "./clinical_notes")  # folder with .json files
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "")
AZURE_OPENAI_KEY = os.getenv("AI_FOUNDRY_API_KEY", "")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

TARGET_PHONE_NUMBER = os.getenv("TARGET_PHONE_NUMBER", "")
ACS_CONNECTION_STRING = os.getenv("ACS_CONNECTION_STRING", "")
ACS_PHONE_NUMBER = os.getenv("ACS_OUTBOUND_CALLER_ID", "")

APP_BASE_URL = os.getenv("APP_BASE_URL", "http://localhost:8000")
TTS_VOICE = os.getenv("TTS_VOICE", "en-US-AvaMultilingualNeural")
COGNITIVE_SERVICES_ENDPOINT = os.getenv("COGNITIVE_SERVICES_ENDPOINT", "")

print("NOTES_PATH:", NOTES_PATH)
print("AZURE_OPENAI_ENDPOINT set:", bool(AZURE_OPENAI_ENDPOINT))
print("AZURE_OPENAI_KEY set:", bool(AZURE_OPENAI_KEY))
print("AZURE_OPENAI_DEPLOYMENT_NAME:", AZURE_OPENAI_DEPLOYMENT_NAME or "(not set)")

### Azure OpenAI Client Initialization
Creates the Azure OpenAI client using environment variables. This is the only place credentials are bound.
Fail-fast validation ensures misconfiguration is caught *before* iterating any patient files.

In [ ]:
# Initialize Azure OpenAI client

from openai import AzureOpenAI

if not AZURE_OPENAI_ENDPOINT or not AZURE_OPENAI_KEY or not AZURE_OPENAI_DEPLOYMENT_NAME:
    raise RuntimeError("Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME.")

client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION  # update if your resource uses a different version
)
print("Azure OpenAI client initialized.")


### Primary SYSTEM_PROMPT Definition
Defines the instruction block for the FIRST model call. Responsibilities:
- Analyze a single patient's clinical JSON.
- Decide if an appointment is needed (`appointment_needed`).
- If needed, emit a richly structured multi-line `CALL_BRIEF` string wrapped between BEGIN/END markers.
The strict JSON envelope keeps downstream parsing trivial and minimizes payload size.

In [ ]:
SYSTEM_PROMPT = r"""
You are an AI health agent that reviews patient history in JSON format.
Your tasks are:
1) Evaluate preventive care needs based on the patient's history.
2) If an appointment is needed, generate a detailed call brief for a voice agent.

Input: You will receive ONE JSON object with patient clinical notes.

Output: ALWAYS return JSON in this exact shape:
{
  "patient_id": "string",
  "appointment_needed": true,
  "call_brief": "BEGIN CALL_BRIEF\nNAME: Jane\nAGE: 62\nAGE_BUCKET_SPOKEN: in your sixties\nSEX: female\nPRONOUNS: she/her\n\nMULTI_NEED: true\nTOP_NEED: colonoscopy\n\nNEEDS:\n- AREA: colonoscopy\n  PRIORITY: high\n  TIMING: in the next one to three months\n  WHY_SHORT: Last colonoscopy was back in October 2013; the ten-year repeat is overdue.\n  HISTORY_SPOKEN: Screening colonoscopy in October 2013; no colonoscopy documented as of March 2021\n  OVERDUE_FLAG: true\n\n- AREA: mammogram\n  PRIORITY: routine\n  TIMING: by August 2024\n  WHY_SHORT: No mammogram in the past two years; recommended every one to two years from forty to seventy-four.\n  HISTORY_SPOKEN: No recent mammogram noted as of March 2021\n  OVERDUE_FLAG: true\n\nINTENT_CLUES: schedule|set up|book|dates|times\nSCHED_WINDOW_PREF: this week or next two weeks\nEND CALL_BRIEF"
}

Rules:
- Be concise and clinically relevant.
- Do not invent data not present in notes.
- If unknown/missing, use "not_documented" or null for string values.
- If nothing is due, return "appointment_needed": false and "call_brief": null.
- The `call_brief` field must be a string containing the full brief as shown in the example, or null.
- The `patient_id` should be extracted from the input JSON.
"""

### Minimal Helper Utilities
Utility functions only:
- `load_json_files` enumerates patient JSON files.
- `read_json` loads one file.
- `try_extract_json` robustly recovers a JSON object from the model output (direct, fenced code block, or best-effort brace scan) to reduce prompt brittleness.
These are intentionally lean—previous heavier helpers were removed to streamline the path from raw file to structured result.

In [ ]:
# Functions for loading/validating notes and parsing model output

def load_json_files(notes_path: str) -> List[Path]:
    p = Path(notes_path)
    if not p.exists() or not p.is_dir():
        raise FileNotFoundError(f"NOTES_PATH does not exist or is not a directory: {notes_path}")
    return sorted([f for f in p.iterdir() if f.suffix.lower() == ".json"])

def read_json(path: Path) -> Any:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def try_extract_json(text: str) -> Optional[dict]:
    """Try to parse JSON from model output: direct, fenced code, or brace scan."""
    # direct
    try:
        return json.loads(text)
    except Exception:
        pass

    # fenced ```json ... ```
    fence = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, flags=re.S|re.I)
    if fence:
        try:
            return json.loads(fence.group(1))
        except Exception:
            pass

    # coarse brace matching
    start, end = text.find("{"), text.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end+1]
        for j in range(len(candidate), 1, -1):
            snippet = candidate[:j]
            try:
                return json.loads(snippet)
            except Exception:
                continue
    return None

### First-Stage Inference Loop
Workflow per patient file:
1. Load raw JSON.
2. Send to model with `SYSTEM_PROMPT`.
3. Attempt robust JSON extraction.
4. Append a normalized result record (never crashes the loop on single-file failure).
Design choices:
- Temperature 0 for determinism.
- Graceful fallback object on parse errors.
- Minimal fields retained to keep memory + downstream logic small.

In [ ]:
# Process each JSON

files = load_json_files(NOTES_PATH)
print(f"Found {len(files)} JSON file(s) in {NOTES_PATH}.")

results = []

for i, path in enumerate(files, start=1):
    try:
        print(f"\n[{i}/{len(files)}] Processing: {path.name}")
        # Simplified: read the json file directly.
        patient_obj = read_json(path)

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": json.dumps(patient_obj, ensure_ascii=False)}
        ]

        resp = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT_NAME,  # your *deployment* name
            messages=messages,
            temperature=0
        )
        content = resp.choices[0].message.content

        parsed = try_extract_json(content)
        if not parsed:
            # If parsing fails, create a minimal error object
            parsed = {
                "patient_id": path.stem, # Use filename stem as a fallback patient_id
                "appointment_needed": False,
                "call_brief": None,
                "error": "Model response was not valid JSON",
                "raw": content
            }

        # The new structure is simpler, so we just append the parsed object
        # along with the source filename.
        results.append({
            "file": path.name,
            "patient_id": parsed.get("patient_id", path.stem),
            "appointment_needed": bool(parsed.get("appointment_needed", False)),
            "call_brief": parsed.get("call_brief"),
            "error": parsed.get("error") # Carry over any errors
        })

    except Exception as e:
        results.append({
            "file": path.name,
            "patient_id": "(unknown)",
            "appointment_needed": False,
            "call_brief": None,
            "error": str(e)
        })

print("\nCompleted.")
len(results)

### Display Raw Call Briefs (Diagnostic)
Iterates accumulated results and prints each `CALL_BRIEF` only for patients flagged `appointment_needed`.
Useful for:
- Quick qualitative spot checks of prompt adherence.
- Ensuring parse fallbacks didn't silently mask issues.
This is a human-facing inspection step—safe to skip in automation.

In [ ]:
# Print the call brief for each patient needing an appointment
for r in results:
    if r.get("appointment_needed"):
        print(f"--- CALL BRIEF for Patient: {r.get('patient_id', 'Unknown')} ---")
        if r.get("call_brief"):
            print(r['call_brief'])
        else:
            print("(No call brief generated)")
        print("-" * 50 + "\n")

### Build & Sort Results DataFrame
Converts the in-memory list of dicts to a tidy DataFrame restricted to the minimal analytic columns.
Sorting brings patients needing outreach to the top for convenient browsing.
This becomes the canonical structured artifact feeding the second model stage.

In [ ]:
df = pd.DataFrame(results)

# Keep only relevant columns
df = df[["file", "patient_id", "appointment_needed", "call_brief", "error"]]

df.sort_values(by=["appointment_needed", "patient_id"], ascending=[False, True], inplace=True, kind="stable")
df.reset_index(drop=True, inplace=True)
df

### Second-Stage CALL_SYSTEM_PROMPT
This prompt transforms a single `CALL_BRIEF` into a patient-facing outbound phone message:
- Enforces tone, length, safety, and formatting (plain text only).
- Treats multi-need vs single-need differently for wording.
It's deliberately concise to reduce token footprint while constraining style.

In [ ]:
CALL_SYSTEM_PROMPT = r"""
You generate one concise, conversational phone message asking a patient to make a MyChart appointment.

Input: A CALL_BRIEF string with patient details and health needs.

Output: Only the raw text message, no JSON, no extra formatting.

Rules:
- Always include the patient’s first name (from the NAME field) in the salutation.
- Begin the message as if it’s coming from an assistant at the Microsoft Health Clinic.
- Tone: conversational and friendly.
- Reason: If MULTI_NEED is false, you can mention the TOP_NEED. If MULTI_NEED is true, keep it generic ("preventive care" or "health needs").
- Length: 15–35 words, max 220 characters, 1–2 sentences.
- Keep language natural, simple, and easy to speak aloud.
- No sensitive details beyond the first name and one preventive item (if included).
- Close with a friendly nudge to confirm or schedule soon.
- Do not include clinic phone numbers or links (these are added later by the dialer).

Example Input:
BEGIN CALL_BRIEF
NAME: Robert
...
TOP_NEED: colonoscopy
MULTI_NEED: false
...
END CALL_BRIEF

→ Example Output:
Hey Robert, it’s us again from the Microsoft Health Clinic. We noticed it’s been a while since your screening colonoscopy — would you like to set up an appointment soon?

Example Input:
BEGIN CALL_BRIEF
NAME: Sarah
...
TOP_NEED: colonoscopy
MULTI_NEED: true
...
END CALL_BRIEF

→ Example Output:
Hi Sarah, this is the Microsoft Health Clinic. It looks like you may be due for your preventive care. Would you like to schedule a visit with us?
"""

### Select Target Patient & Assemble Second Call Messages
Filters only patients needing outreach, selects the second one (demo logic), and constructs the chat message list (`call_prompt`) for the second model call.
If insufficient patients qualify, it safely aborts by leaving `call_prompt` empty.

In [ ]:
# Prepare call prompt for the second patient in the dataframe

# Filter for patients who need an appointment
outreach_df = df[df["appointment_needed"] == True].copy()

if len(outreach_df) > 1:
    # Get the call_brief from the second patient in the filtered dataframe
    call_brief_string = outreach_df.iloc[1]["call_brief"]

    call_prompt = [
        {"role": "system", "content": CALL_SYSTEM_PROMPT},
        {"role": "user", "content": call_brief_string}
    ]

    print("--- Using Call Brief for Prompt ---")
    print(call_brief_string)
else:
    print("Not enough patients needing an appointment to select the second one.")
    call_prompt = [] # Ensure call_prompt is empty so the next cell doesn't run


### Generate Outbound Call Message Text
Executes the second model call using the prepared `call_prompt` to obtain a final, human-spoken style message.
Output is plain text only—ready for text-to-speech and dialing in subsequent steps.

In [ ]:
# Make the call message to Azure OpenAI

resp = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_NAME,  # your *deployment* name
    messages=call_prompt,
    temperature=0
)
call_message = resp.choices[0].message.content

print(call_message)

## Make Phone Call ##

### Place Phone Call via Azure Communication Services
The next code cell dials the target number using ACS, waits for connection, plays the synthesized message using the configured neural voice, then hangs up after a short dwell window.
Includes a timeout guard and conservative sleep logic to avoid premature hang-up.

> **Demo Only (One‑Way Playback):** This cell places a simple outbound call and plays the generated `call_message`. 
> It does not listen for speech/DTMF, gather patient input, or branch logic. 
> For the interactive / production workflow (turn-taking, state, routing), see `main.py`. 

In [ ]:
import os
import time
from azure.communication.callautomation import (
    CallAutomationClient,
    PhoneNumberIdentifier,
    TextSource,
    CallConnectionState,
)


def call_and_say(message: str) -> None:
    acs = CallAutomationClient.from_connection_string(ACS_CONNECTION_STRING)

    create_result = acs.create_call(
        target_participant=PhoneNumberIdentifier(TARGET_PHONE_NUMBER),   # E.164 format
        callback_url=APP_BASE_URL,                                       # must be publicly reachable HTTPS
        source_caller_id_number=PhoneNumberIdentifier(ACS_PHONE_NUMBER), # caller ID shown to callee
        #cognitive_services_endpoint=os.environ["COGNITIVE_SERVICES_ENDPOINT"],
        cognitive_services_endpoint=COGNITIVE_SERVICES_ENDPOINT
    )

    call_conn = acs.get_call_connection(create_result.call_connection_id)

    # Wait briefly for the call to connect before playing media
    deadline = time.time() + 30
    while time.time() < deadline:
        state = call_conn.get_call_properties().call_connection_state
        if state == CallConnectionState.CONNECTED or str(state).lower() == "connected":
            break
        time.sleep(0.5)
    else:
        raise TimeoutError("Call did not connect within 30 seconds.")

    # Speak the message using your chosen voice
    tts_voice = TTS_VOICE
    text_source = TextSource(text=message, voice_name=tts_voice)
    call_conn.play_media_to_all(text_source)

    # Optional: hang up after a short delay
    time.sleep(min(15, max(5, len(message) / 10)))
    call_conn.hang_up(is_for_everyone=True)

call_and_say(call_message)